In [1]:
from numba import jit
from PIL import Image
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import random
%matplotlib inline
import pandas as pd
from statsmodels.tsa.stattools import acf
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Details of model

In [ ]:
N    = 10 #dimensions of matrix
J1   = 2.3*10**(-3) #coupling constant in Ev
J2   = -21*10**(-3)
H    = 0 #magnetic field, must be set to 0 to compute observables 
counts = 1000
T    = (0.01) #temperature in K
k    = 8.6173303*10**(-5) #boltzmann constant in Ev/K
beta = 1/(k*T)

In [ ]:
def initial_state_Nio(N): 
    '''initialize the lattice'''
    state = np.random.choice([-1, 1], (N, N))
    state[::2, ::2] = 0
    state[1::2, 1::2] = 0
    return state
def diag_nbrs(i,j,N): 
    return [((i+1)%N,(j+1)%N),((i+1)%N,(j-1)%N),((i-1)%N,(j+1)%N),((i-1)%N,(j-1)%N)]
def lat_nbrs(i,j,N): 
    return [(i,(j+2)%N),(i,(j-2)%N),((i+2)%N, j),((i-2)%N, j)]


In [ ]:
def Energy_Nio(state, J1, J2, H):
    J1   = 2.3*10**(-3) #diagonal
    J2   = -21*10**(-3) #lateral coupling
    E = 0
    N = state.shape[0]
    for x in range(N):
        for y in range(N):
            if (x-y)%2: # (x-y)%2 == 0 are oxygen atoms with no interaction
                nbrs = diag_nbrs(x,y,N)
                for nbr in nbrs:
                    E += -state[x,y]*J1*state[nbr[0],nbr[1]]
                nbrs = lat_nbrs(x,y,N)
                for nbr in nbrs:
                    E += -state[x,y]*J2*state[nbr[0],nbr[1]]
    E/=2
    E -= H*state.sum()
    return E

In [ ]:
def calcMag(state):
    return np.sum(state)

def step_update_Nio(state, beta, J1, J2,H,energy,mag,N):
    J1   = 2.3*10**(-3) #diagonal
    J2   = -21*10**(-3) #lateral coupling
    for i in range(N**2): #1 step per state on average
        dE = 0
        x = random.randint(0,N-1)
        y = random.randint(0,N-1)
        if (x-y)%2:
            nbrs = diag_nbrs(x,y,N)
            for nbr in nbrs:
                dE += 2*state[x,y]*J1*state[nbr[0],nbr[1]]
            nbrs = lat_nbrs(x,y,N)
            for nbr in nbrs:
                dE += 2*state[x,y]*J2*state[nbr[0],nbr[1]]
            dE += 2*H*state[x,y]
            if (dE <= 0):
                if state[x,y] == 1:
                    mag-=2
                else:
                    mag+=2
                energy += dE
                state[x, y] *= -1
            else:
                r = random.uniform(0,1)
                tau = np.exp(-dE*beta)
                if (r < tau) :
                    if state[x,y] == 1:
                        mag-=2
                    else:
                        mag+=2
                    energy += dE
                    state[x, y] *= -1
    return state,energy,mag

def run_Nio(state, steps, N, beta, J1, J2,H):
    '''runs for steps times'''
    J1   = 2.3*10**(-3) 
    J2   = -21*10**(-3)
    E = np.zeros(steps)
    M = np.zeros(steps)
    energy = Energy_Nio(state, J1, J2,H)
    mag = calcMag(state)
    for i in range(steps):
        state,energy,mag = step_update_Nio(state, beta, J1, J2,H,energy,mag,N)
        E[i] = energy
        M[i]= mag
#     plt.plot(E)
#     plt.show()
#     plt.plot(M)
#     plt.show()
    return state,E,M

### running for variable temperatures

In [ ]:
N    = 12 #dimensions of matrix
J1   = 2.3*10**(-3) #coupling constant in Ev
J2   = -21*10**(-3)
H    = 0 #magnetic field, must be set to 0 to compute observables 
counts = int(1e5)
T_start = np.log(0.0001)
T_end = np.log(1e6)
T_cnt = 10
# T    = (0.01) #temperature in K
Ts = np.logspace(T_start,T_end,T_cnt)
k    = 8.6173303*10**(-5) #boltzmann constant in Ev/K
# beta = 1/(k*T)
states = np.zeros(shape=(T_cnt,N,N))
Ener1s,Mag1s = np.zeros(shape = (T_cnt,counts)), np.zeros(shape = (T_cnt,counts))

for i in range(Ts.size):
    T = Ts[i]
    beta = 1/(k*T)
    state = initial_state_Nio(N)
    # counts = 10000
    state,E,M = run_Nio(state, counts, N, beta, J1, J2, H)

    print (state)
    Ener1s[i] = E
    Mag1s[i] = M
    plt.imshow(state,cmap='bwr',interpolation="none") 
    plt.show() #plots last configuration